# Florida Worker Adjustment & Retraining Notifications

Companies meeting certain requirements are required to file notices with the state about workers that are being laid off, furloughed or having hours cut. Those filings can be accessed at:

https://floridajobs.org/office-directory/division-of-workforce-services/workforce-programs/reemployment-and-emergency-assistance-coordination-team-react/warn-notices

## Aggregating and cleaning WARN data

### Step 1

First, I will import the requests and BeautifulSoup libraries, so that I can quickly retrieve and filter the HTML code from the state's WARN website. 

Then, I will make a GET request of the website and retrieve the retrieved HTML code.

In [1]:
import requests
from bs4 import BeautifulSoup
url = 'http://reactwarn.floridajobs.org/WarnList/Records?year=2020'
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text)

### Step 2

I will find out the number of webpages to loop through. I will store each page number in a list, which I will use to loop through and append to the end of my subsequent get requests.

In [2]:
pages = []
for page in range(1,15):
    if len(pages) != len([]):
        if page > int(pages[-1]):
            break
    url = f'http://reactwarn.floridajobs.org/WarnList/Records?year=2020&page={page}'
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text)
    lst = soup.tfoot.td.text.split()
    for item in lst:
        if item == '>' or item == '<':
            pass
        else:
            if item not in pages:
                pages.append(item)
            else:
                continue
print(pages)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13']


### Step 3

Now I will loop through every page, and extract the data into a series of lists.

In [3]:
company = []
address = []
zip_code = []
city = []
notification_date = []
start_date = []
end_date = []
employees = []
industry = []
links = []

for page in pages:
    url = f'http://reactwarn.floridajobs.org/WarnList/Records?year=2020&page={page}'
    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text)
    tds = soup.tbody.find_all('td')
    
    for i in range(0,len(tds),6):
        company.append(str(tds[i])[7:-5].split("</b> ")[0])
        address.append(str(tds[i])[7:-5].split("</b> ")[1].split(", FL, ")[0])
        zip_code.append(str(tds[i])[7:-5].split("</b> ")[1].split(", FL, ")[1])
    
    for i in range(1,len(tds),6):
        notification_date.append(str(tds[i])[4:6]+'/'+str(tds[i])[7:9]+'/'+str(tds[i])[10:12])
        
    for i in range(2,len(tds),6):
        start_date.append(str(tds[i])[4:6]+'/'+str(tds[i])[7:9]+'/'+str(tds[i])[10:12])
        end_date.append(str(tds[i]).split("i> ")[2][:2]+'/'+str(tds[i]).split("i> ")[2][3:5]+'/'+str(tds[i]).split("i> ")[2][6:-5])
    
    for i in range(3,len(tds),6):
        employees.append(int(str(tds[i])[4:-5]))
        
    for i in range(4,len(tds),6):
        industry.append(str(tds[i])[4:-5])
        
    for i in range(5,len(tds),6):
        try:
            links.append('http://reactwarn.floridajobs.org'+str(tds[i]).split('''="''')[1][:-19])
        except:
            links.append('None')

for a in address:
    reverse = str()
    for i in reversed(range(0,len(a))):
        reverse = reverse + a[i]
    counter = 0
    for i in range(0,len(reverse)):
        if (reverse[i].isupper() and reverse[i] not in [0,1,2,3,4,5,6,7,8,9] and reverse[i] != ".") or reverse[i] == " ":
            counter += 1
        else:
            idx = counter * -1
            break
    if a[idx] == ' ':
        if a[idx-3:idx].upper() == "ST.":
            idx = idx - 3
        else:
            idx = idx + 2
    city.append(a[idx:].title())

# Fix common errors
for i in range(0,len(city)):
    if city[i] == 'Aboynton Beach':
        city[i] = 'Boynton Beach'
    elif city[i] == 'Acoral Springs':
        city[i] = 'Coral Springs'
    elif city[i] == 'Amiami':
        city[i] = 'Miami'
    elif city[i] == 'Aorange Park':
        city[i] = 'Orange Park'
    elif city[i] == 'Aport Saint Lucie':
        city[i] = 'Port Saint Lucie'
    elif city[i] == 'Asatellite Beach':
        city[i] = 'Satellite Beach'
    elif city[i] == 'Awinter Park':
        city[i] = 'Winter Park'
    elif city[i] == 'Bfort Lauderdale':
        city[i] = 'Fort Lauderdale'
    elif city[i] == 'Awinter Park':
        city[i] = 'Winter Park'
    elif city[i] == 'Ckendall':
        city[i] = 'Kendall'
    elif city[i] == 'Npalm Harbor':
        city[i] = 'Palm Harbor'
    elif city[i] == 'St. Nnaples':
        city[i] = 'Naples'    
for i in range(0,len(address)):
    address[i] = address[i][:-len(city[i])]    


### Step 4

Now I will append my lists into a Pandas dataframe.

In [4]:
import pandas as pd
df = pd.DataFrame(list(zip(company,address,zip_code,city,notification_date,start_date,end_date,employees,industry,links)),
            columns=['company','address','zip','city','notification_date','start_date','end_date','employees','industry','filing'])
df.head(10)

,company,address,zip,city,notification_date,start_date,end_date,employees,industry,filing
0,TrueCore Behavioral Solutions,5050 N.E. 168th St.,34972,Okeechobee,10/08/20,09/28/20,09/28/20,101,Administrative and Support and Waste Managemen...,http://reactwarn.floridajobs.org/WarnList/Down...
1,"Cuba Libre Orlando, LLC t/a Cuba Libre Restaur...",9101 International Dr.,32819,Orlando,10/08/20,10/10/20,10/10/20,120,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
2,Hyatt Regency Orlando,9801 International Dr.,32819,Orlando,10/12/20,10/17/20,10/17/20,315,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
3,ABM. Inc.,Nova Southeastern University3301 College Ave.,33314,Fort Lauderdale,10/12/20,11/30/20,11/30/20,269,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
4,Newport Beachside Resort,16701 Collins Avenue,33160,Sunny Isles Beach,10/13/20,11/30/20,11/30/20,100,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
5,Homewood Suites by Hilton Orlando-Internationa...,8745 International Drive,32819,Orlando,10/15/20,11/17/20,11/17/20,22,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
6,Embassy Suites by HIlton Orlando International...,8250 Jamaican Court,32819,Orlando,10/15/20,11/17/20,11/17/20,20,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
7,Hilton Singer Island Oceanfront/Palm Beaches R...,3700 North Ocean Drive,33404,Riviera Beach,10/15/20,11/17/20,11/17/20,19,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
8,Wyndham Vacation Ownership/The Bonnet Creek Re...,9560 Via Encinas,32830,Orlando,10/14/20,09/04/20,10/17/20,6,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...
9,"Dr. Smood, LLC",Aventura19501 Biscayne Blvd.,33180,Miami,10/13/20,11/04/20,11/04/20,5,Accommodation and Food Services,http://reactwarn.floridajobs.org/WarnList/Down...


### Step 4

Finally, I will save this dataframe to a Google Sheet so that it can be easily shared.


###### Note: Follow these instructions to be able to save to your own Google Sheet: 
https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c

In [5]:
# !pip install google_spreadsheet
# !pip install google-auth-oauthlib

from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize

In [6]:
import google_spreadsheet
import google_auth_oauthlib
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle

In [7]:
#change this to your sheet ID
gsheetId = ''

#change the range if needed
SAMPLE_RANGE_NAME = 'A1:AA10000'

def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
    global service
    SCOPES = [scope for scope in scopes[0]]
    #print(SCOPES)
    
    cred = None

    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
            cred = flow.run_local_server()

        with open('token_write.pickle', 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(api_service_name, api_version, credentials=cred)
        print(api_service_name, 'service created successfully')
        #return service
    except Exception as e:
        print(e)
        #return None
        
# change './credentials.json' to your downloaded JSON file.
Create_Service('./credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])
    
def Export_Data_To_Sheets():
    response_date = service.spreadsheets().values().update(
        spreadsheetId=gsheetId,
        valueInputOption='RAW',
        range=SAMPLE_RANGE_NAME,
        body=dict(
            majorDimension='ROWS',
            values=df.T.reset_index().T.values.tolist())
    ).execute()
    print('Sheet successfully Updated')

Export_Data_To_Sheets()

sheets service created successfully
Sheet successfully Updated


Here is the final spreadhseet:
    
https://docs.google.com/spreadsheets/d/14ZRbs59AqFmJsY6Aehu0HtBzqIyWO7L6rWSG4rbYB38/edit#gid=0

And here are some of the articles that utilized this data:
    
- https://www.bizjournals.com/jacksonville/news/2020/07/17/downtown-jaguars-jobs-jacksonville-layoffs.html
- https://www.bizjournals.com/jacksonville/news/2020/05/14/ponte-vedra-layoffs-northeast-florida-covid19.html
- https://www.bizjournals.com/jacksonville/news/2020/06/15/downtown-ponte-vedra-see-hundreds-of-hotel-layoffs.html